In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import math
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

# K-Means

In [ ]:
df = pd.read_csv("mushroom.csv")

In [ ]:
df[df["class"] == "p"]["class"].count()

In [ ]:
df[(df["class"] == "p")]["cap-diameter"].max()

In [ ]:
df[(df["stem-height"] <= 21) & (df["stem-width"] <= 60) & (df["cap-diameter"] <= 31) & (df["class"] == "e")]["class"].count()

In [ ]:
!pip install dataprep
from dataprep.eda import plot, plot_correlation, create_report, plot_missing

In [ ]:
df.describe()

In [ ]:
create_report(df)

In [ ]:
df = df.drop(columns = ["spore-print-color", "veil-color", "veil-type", "stem-root", "stem-surface"])

In [ ]:
df.info()

In [ ]:
dffloat = df[["cap-diameter", "stem-height", "stem-width", "class"]]

In [ ]:
dffloat.info()

In [ ]:
def distance(x, y):
    squaresum = 0
    for i in range(len(x)):
        squaresum = squaresum + (x[i] - y[i])**2
    return np.sqrt(squaresum)

In [ ]:
def kmeans(df, k=4, tol=0.05): 
    """
    Usage: input 
        df=data frame, 
        k=# of clusters
        tol=tolerance for L_2 convergance check on centroids
    """    
    #Do the thing
    X = df.values
    centroids = X[np.random.choice(X.shape[0], size=k, replace=False)]
    clusters = np.array([-1]*len(X))
    meanerror = float('inf')
    while True:
        for i in range(len(clusters)):
            dists = [distance(X[i], cent) for cent in centroids]
            assignment = dists.index(min(dists))
            clusters[i] = assignment

        new_centroids = np.array([X[clusters == j].mean(axis=0) for j in range(k)])
        
        new_meanerror = np.mean([distance(X[i], new_centroids[clusters[i]])**2 for i in range(len(X))])
        
        if abs(meanerror - np.mean(new_meanerror)) <= tol:
            break
            
        centroids = new_centroids
        meanerror = new_meanerror
    return centroids, clusters, meanerror 

In [ ]:
centroids, clusters, meanerror = kmeans(dffloat[["cap-diameter", "stem-height", "stem-width"]], k=5)

In [ ]:
listmeanerror = []
minmeanerror = 10000000
leasterror = 0
for i in range(1, 15):
    print(i)
    centroids, clusters, meanerror = kmeans(dffloat[["cap-diameter", "stem-height", "stem-width"]], k=i)
    listmeanerror.append(meanerror)
    if meanerror < minmeanerror:
        leasterror = i
        

In [ ]:
plt.plot(range(1, 15), listmeanerror)
plt.title("Error plot for K-Means")
plt.xlabel("Number of Clusters")
plt.ylabel("Mean Error")

In [ ]:
dffloat[["cap-diameter", "stem-height", "stem-width"]]

### K-Means on stem height and width and cap diameter
I found that theres only a certain range for all numerical features that mushrooms are poisonous

In [ ]:
plt.scatter(dffloat["stem-height"].values, dffloat["stem-width"].values, c=clusters)
# plt.scatter(dffloat[dffloat["class"] == "p"]["stem-height"].values, dffloat[dffloat["class"] == "p"]["stem-width"].values, c="blue", s=0.01)
plt.xlabel("Stem-Height")
plt.ylabel("Stem Width")
plt.scatter([x[1] for x in centroids], [x[2] for x in centroids], c='red', marker='+')
plt.title("2 Clusters with K-Means")

In [ ]:
for i in range(0, 5):
    x = dffloat[(clusters == i) & (dffloat["class"] == "p")]["class"].count() / dffloat[clusters == i]["class"].count()
    print(max(x, 1-x))

In [ ]:
clusters == 1

In [ ]:
plt.scatter(dffloat[dffloat["class"] == "e"]["stem-height"].values, dffloat[dffloat["class"] == "e"]["stem-width"].values, label="Edible")
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-height"].values, dffloat[dffloat["class"] == "p"]["stem-width"].values, label="Poisonous")
plt.legend()
plt.xlabel("Stem-Height")
plt.ylabel("Stem Width")
plt.title("P vs E")

In [ ]:
plt.scatter(dffloat["stem-height"].values, dffloat["cap-diameter"].values, c=clusters)
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-height"].values, dffloat[dffloat["class"] == "p"]["cap-diameter"].values, c="blue", s=0.01)

In [ ]:
plt.scatter(dffloat["stem-width"].values, dffloat["cap-diameter"].values, c=clusters)
#plt.scatter(dffloat[dffloat["class"] == "p"]["stem-width"].values, dffloat[dffloat["class"] == "p"]["cap-diameter"].values, c="blue", s=0.01)
plt.xlabel("Stem-Width")
plt.ylabel("Cap-Diameter")
plt.scatter([x[2] for x in centroids], [x[0] for x in centroids], c='red', marker='+')
plt.title("5 Clusters with K-Means")

In [ ]:
plt.scatter(dffloat[dffloat["class"] == "e"]["stem-width"].values, dffloat[dffloat["class"] == "e"]["cap-diameter"].values, label="Edible")
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-width"].values, dffloat[dffloat["class"] == "p"]["cap-diameter"].values, label="Poisonous")
plt.legend()
plt.xlabel("Stem Width")
plt.ylabel("Cap Diameter")
plt.title("P vs E")

### Pruning dataset to only certain numerical ranges of features

In [ ]:
dffloat = dffloat[(dffloat["class"] == "p") & (dffloat["cap-diameter"] <= 31)]

In [ ]:
dffloat = dffloat[(dffloat["class"] == "p") & (dffloat["stem-height"] <= 21)]

In [ ]:
dffloat = dffloat[(dffloat["class"] == "p") & (dffloat["stem-width"] <= 60)]

In [ ]:
centroids, clusters, meanerror = kmeans(dffloat[["cap-diameter", "stem-height", "stem-width"]], k=4)

In [ ]:
plt.scatter(dffloat["stem-height"].values, dffloat["stem-width"].values, c=clusters)
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-height"].values, dffloat[dffloat["class"] == "p"]["stem-width"].values, c="blue", s=0.01)

In [ ]:
plt.scatter(dffloat["stem-height"].values, dffloat["cap-diameter"].values, c=clusters)
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-height"].values, dffloat[dffloat["class"] == "p"]["cap-diameter"].values, c="blue", s=0.01)

In [ ]:
plt.scatter(dffloat["stem-width"].values, dffloat["cap-diameter"].values, c=clusters)
plt.scatter(dffloat[dffloat["class"] == "p"]["stem-width"].values, dffloat[dffloat["class"] == "p"]["cap-diameter"].values, c="blue", s=0.01)

In [ ]:
create_report(dffloat)

In [ ]:
create_report(df)

# GMM

In [ ]:
df=pd.read_csv('mushroom.csv', encoding='UTF-8')
print(df[['cap-diameter', 'stem-height','stem-width']])

In [ ]:
def GMM_starter(dat, k):
    np.random.seed(70)
    
    mushrooms = dat.values
    
    #This represents Σ_m (covariance matrix of each component)
    covars=np.zeros((k,3,3))
    
    #This represents μ_m (mean values of each component)
    means=np.zeros((k,3))
    
    #This represents W_m (weights/likelihood of each component)
    p_class=np.zeros(k)
    
    #This represents P(x_i | cluster = m), or Φ(x_i | μ_m, Σ_m)
    p_data_given_class=np.zeros((len(dat),k))
    p_class_given_data=np.zeros((len(dat),k))

    
    """Initialize means, covs, p_classes"""
    #initializations of starting points (used to set the initial means below)
    init_idx=np.random.choice(range(len(dat)), size=k, replace=False)
    
    #Initialize the covariance matrix, the means, and the p_class for each of k components (dims)
    for dim in range(k):
        #Set the cov matrix of each component to the cov of the entire dataset
        covars[dim,:,:]=np.cov(np.transpose(dat))

        #Set initial means to initial chosen data points
        means[dim,:]=dat.iloc[init_idx[dim]]
        
        #Give each component equal weighting / likelihood to start
        p_class[dim]=1/k
        
    for step in range(50):  
        
        #Expectation Step
        summ = np.zeros(len(dat))
        for i in range(k):
            p_data_given_class[:, i] = stats.multivariate_normal.pdf(mushrooms, mean=means[i], cov=covars[i])
            p_class_given_data[:, i] = p_data_given_class[:, i] * p_class[i]
            summ += p_class_given_data[:, i]
        
        for i in range(k):
            p_class_given_data[:, i] /= summ
            
        
        #Maximization Step
        for i in range(k):
            # updating weights
            count = np.sum(p_class_given_data[:, i])
            p_class[i] = count/len(dat)
            
            #updating means
            for p in range(3):
                means[i][p] = (1/count) * np.sum(p_class_given_data[:, i] * mushrooms[:,p])
            
            #updating variances
            covars[i] = np.cov(np.transpose(dat),aweights=p_class_given_data[:, i], ddof=0)
    
    mean_dist = 0
    for i in range(k):
        for p in range(3):
            mean_dist += np.sum(p_class_given_data[:, i] * ((mushrooms[:,p]-means[i][p])**2))
    mean_dist = np.sqrt(mean_dist/len(dat)/k)
    
    #Return all the needed variables
    return p_class_given_data, means, covars, p_class, mean_dist

Below we run an elbow plot to see mean errors for each value of k from 1 to 8.

In [ ]:
# graphing mean distances for k, 1 to 8
X = range(1,9)
mean_distances = []
for k in range(1,9):
    p_class_given_data, means, covars, p_class, mean_dist = GMM_starter(df[["cap-diameter","stem-height",'stem-width']], k)
    mean_distances.append(mean_dist)
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(X, mean_distances)
plt.xlabel("k-value")
plt.title("Elbow Plot of Weighted Mean Distance")
plt.ylabel("Distance")
plt.show()

Below, we start comparing plots of our formed clusters (based on cap diameter, stem height, and stem width) and comparing the clusters to the poisinous vs edible to see if the clusters are correct.

In [ ]:
p_class_given_data, means, covars, p_class, mean_dist = GMM_starter(df[["cap-diameter","stem-height",'stem-width']], 2)

# graphing the points/clusters
dataPoints = df[["stem-height",'stem-width']].values
colors = ['blue','yellow']
colorsIndex = np.argmax(p_class_given_data, axis=1)
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(dataPoints[:,0], dataPoints[:,1],c=[colors[i] for i in colorsIndex])
plt.title("2 clusters")
plt.xlabel("Stem Height")
plt.ylabel("Stem Width")
plt.show()

#graphing the points/actual classifications
dataPoints = df[["stem-height",'stem-width']].values
colors = ['blue','yellow']
colorsIndex = df["class"] == "p"
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(dataPoints[:,0], dataPoints[:,1],c=[colors[i] for i in colorsIndex])
plt.title("P vs E")
plt.xlabel("Stem Height")
plt.ylabel("Stem Width")
plt.show()

Above, we have made 2 plots. One of the 2 clusters and the other plot is edible vs poisinous. We can see that the clusters don't represent the classification very well.

In [ ]:
p_class_given_data, means, covars, p_class, mean_dist = GMM_starter(df[["cap-diameter","stem-height",'stem-width']], 8)

# graphing the points/8 clusters
dataPoints = df[["stem-height",'stem-width']].values
colors = ['red','green','blue','yellow','pink','brown','orange','purple','grey']
colorsIndex = np.argmax(p_class_given_data,axis=1)
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(dataPoints[:,0], dataPoints[:,1],c=[colors[i] for i in colorsIndex])
plt.title("8 clusters")
plt.xlabel("Stem Height")
plt.ylabel("Stem Width")
plt.show()

# graphing the points/actual classifications
dataPoints = df[["stem-height",'stem-width']].values
colors = ['blue','yellow']
colorsIndex = df["class"] == "p"
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(dataPoints[:,0], dataPoints[:,1],c=[colors[i] for i in colorsIndex])
plt.title("P vs E")
plt.xlabel("Stem Height")
plt.ylabel("Stem Width")
plt.show()

# graphing the points/clusters with adjusted coloring to look like classification graph
dataPoints = df[["stem-height",'stem-width']].values
colors = ['blue','yellow','yellow','blue','yellow','blue','blue','yellow','grey']
colorsIndex = np.argmax(p_class_given_data,axis=1)
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(dataPoints[:,0], dataPoints[:,1],c=[colors[i] for i in colorsIndex])
plt.title("8 clusters matched with class")
plt.xlabel("Stem Height")
plt.ylabel("Stem Width")
plt.show()

Above we have made 3 plots. The first plot shows the 8 clusters formed. The second plot shows poisinous vs edible. And the last plot, we try to match the 8 clusters with the appropriate poisinous or edible classification. We can see that it is better but not perfect. Now below, we check how much each cluster is either entirely poisinous or entirely edible.

In [ ]:
p_class_given_data, means, covars, p_class, mean_dist = GMM_starter(df[["cap-diameter","stem-height",'stem-width']], 8)
clusterPercentage = []
clusters = np.argmax(p_class_given_data,axis=1)
classes = df["class"].values
# getting the percentages of either p or e for each cluster (picks the higher percentage)
for i in range(8):
    p = np.sum(classes[clusters == i] == "p")
    e = np.sum(classes[clusters == i] == "e")
    clusterPercentage.append(p/(p+e) if p > e else e/(p+e))
    
clusterPercentage

This is an array of the percentages associated with how much the data points in each cluster align with either only poisnous or only edible. For example, the first index is 51.9% meaning that the data points that are mainly in cluster 1, 51.9% of the data points are either only poisinous or are only edible. The point of this test is to see how strong of a correlation these clusters have with the classifications. If all of these values were 1, that would mean each cluster would have only poisionous mushrooms or only edible mushrooms. That would be the ideal as we want the clusters to accurately represent edibile vs poisionous. Note this is for 8 clusters. We now take the average of this percentage and compare the average among different number of clusters.

In [ ]:
kmax = 12
X = range(1,kmax)
mean_percentages = []
for k in range(1,kmax):
    p_class_given_data, means, covars, p_class, mean_dist = GMM_starter(df[["cap-diameter","stem-height",'stem-width']], k)
    clusterPercentage = []
    clusters = np.argmax(p_class_given_data,axis=1)
    classes = df["class"].values
    # getting the percentages of either p or e for each cluster (picks the higher percentage)
    for i in range(k):
        p = np.sum(classes[clusters == i] == "p")
        e = np.sum(classes[clusters == i] == "e")
        clusterPercentage.append(p/(p+e) if p > e else e/(p+e))
            
    mean_percentages.append(sum(clusterPercentage)/len(clusterPercentage))
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.scatter(X, mean_percentages)
plt.xlabel("k-value")
plt.ylabel("Average Cluster Percentage")
plt.show()

We now run the same percentages test on 1 to 11 clusters but instead take the average of the cluster percentages from the previous cell. As we can see from this, the highest percentage was when there was 5 clusters, meaning that at 5 clusters, the clusters on average were either 68% poisionous or 68% edible. Like we previously said, this number should ideally be 1 as each cluster should only be poisinous or only be edible. Likewise, increasing the number of clusters lowers this average percentage even more. This shows that GMM with these 3 number fields is not very good as we want the clusters to mainly be either poisinous or edible, but the best percentage we got was 68%.